# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [0]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        self.n_classes = len(np.unique(y))
        self.y = y
        
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        изучите работу sklearn.neighbors.NearestNeighbors'''
        
        neigh_dist, neigh_ind = self.NearestNeighbors.kneighbors(X)
        
        if self.weights == 'uniform':
            w = np.ones(neigh_dist.shape)
        else:
            w = 1 / (neigh_dist + 1e-3)
        

        pred_labels = self.y[neigh_ind]   
        n_samples = X.shape[0]
        probs = np.zeros((n_samples, self.n_classes))

        all_rows = np.arange(X.shape[0])
        for i, idx in enumerate(pred_labels.T):
            probs[all_rows, idx] += w[:, i]

        normalizer = probs.sum(axis=1)[:, np.newaxis]
        probs /= normalizer
        
        return probs

# Загрузим данные и обучим классификатор

In [0]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, )

Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [54]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

[ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]


Несколько примеров, на которых можно проверить правильность реализованного метода:

In [55]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

68 [0.         0.99816311 0.00183689]
77 [0.         0.99527902 0.00472098]
146 [0.         0.00239145 0.99760855]


**Примечание:** отличие в третьем-четвертом знаке после запятой в тестах не должно повлиять на сдачу задания

# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [56]:
for i in 56, 83, 127:
    print('{:.2f}'.format(max(prediction[i])))

1.00
0.99
1.00
